In [80]:
######################
#Brief:
    # 5 layer DNN ,128 neurons per layer
    # implement:
    #  batch input(for GPU optimize)
    #  N-fold cross validation
    #  print accuacy,precision,recall
    #  Dropout
    #  Optimize:
    #  batch_norm,specify momentum parameter of Adamoptimizer
#input:
    # mnist hand writing dataset 28*28 image(0~5 only)
    # output: 0~5 predicted number
#Result:
    # We found that 5 layers is unnecessary,the network can 
    # perform well (faster) with only 2 layers
#Training process:
    #input 0~5 hand writting image and it's labels from MNIST
    #image will be flatten to 1x784 array 
    #we preprocess the input with batch norm and feed them into 5 layers DNN 
    #Since ELU activation func is applied ,we use He initialize can prevent gradient from vanish
    #The softmax function will highlight the maximun output and also keep some information of other
    #which will make backward propogation faster and also ensure no weight become starvation
    #The dropout technic will prevent lazy neurons
    #We analyze accuracy,precision,recall by calculating the TP FP TN FN of each label
    #The N-fold cross validation provide us a more reliable way to analyze data
####################
import numpy as np
import os
import tensorflow as tf
from tensorflow.contrib.framework import arg_scope
from tensorflow.contrib.layers import fully_connected, batch_norm
from scipy import io
A = abs(io.loadmat('Xia_A.mat')['a_train']);
I = abs(io.loadmat('Xia_I.mat')['i_label']);

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

# training on MNIST but only on digits 0 to 4
X_train1 = np.transpose(A[:,0:150])
y_train1 = np.transpose(I[:,0:150])

print(X_train1.shape)
X_valid1 = np.transpose(A[:,151:160])
y_valid1 = np.transpose(I[:,151:160])

X_test1 = np.transpose(A[:,161:170])
y_test1 = np.transpose(A[:,161:170])
###### Do not modify here ###### 

#get next batch in order
def next_batch(batch_size,iteration, data, labels):
    start = batch_size*iteration
    end = batch_size*(iteration+1)
    return data[start:end], labels[start:end]

#const parameters
n_inputs = 512  
n_outputs = 512

#adjustable parameters
N_neurons = 256
learning_rate = 0.01
momentum = 0.25
epochs = 100
batch_size = 128  #for GPU optimize
dropout = 0.2

# Create the model
X = tf.placeholder(tf.float64, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float64, shape=(None), name="y")
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')


he_init = tf.contrib.layers.variance_scaling_initializer()
#improving traning speed
batch_norm_params = {
    'is_training': is_training,
    'decay': 0.9,
    'updates_collections': None,
    'scale': True,
}


#5 fully connected layer ,128 neurons per layer with dropout
with arg_scope(
        [fully_connected],
        activation_fn=tf.nn.elu,
        weights_initializer=he_init,
        normalizer_fn=batch_norm,
        normalizer_params=batch_norm_params):
    W1 = fully_connected(X,N_neurons)
    W1_D = tf.nn.dropout(W1,dropout)
    W2 = fully_connected(W1_D,N_neurons)
    W2_D = tf.nn.dropout(W2,dropout)

    y_hat = fully_connected(W2_D, n_outputs, activation_fn=None)


#Add softmax to output

my_logits = tf.layers.dense(y_hat,n_outputs,kernel_initializer = he_init,name='logits')
# y_proba = tf.nn.softmax(logits ,name  = 'y_proba')
# cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=my_logits)
# loss = tf.reduce_mean(cross_entropy, name="loss")

loss = tf.reduce_sum(tf.pow(y - y_hat, 2))
#use AdamOptimizer
optimizer = tf.train.AdamOptimizer(learning_rate, momentum)
training_op = optimizer.minimize(loss,name = "training_op")

accuracy =tf.reduce_sum(tf.pow(y - y_hat, 2))/(512*batch_size)


init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    print('restore model')
    saver.restore(sess, "./model/vowel_transfer.ckpt")

    for epoch in range(epochs):
        for iteration in range(len(X_train1)//batch_size):
            X_batch, y_batch = next_batch(batch_size,iteration,X_train1, y_train1)
            sess.run(training_op, feed_dict={is_training: True, X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={is_training: False, X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={is_training: False, X:  X_valid1, y:  y_valid1})
        print(epoch, "Train accuracy:", acc_train, "Valid accuracy:", acc_valid)
    save_path = saver.save(sess, "./model/vowel_transfer.ckpt")
    acc_test = accuracy.eval(feed_dict={is_training: False, X:  X_test1, y:  y_test1})
    print( "Test accuracy:", acc_test)

(150, 512)
restore model
INFO:tensorflow:Restoring parameters from ./model/vowel_transfer.ckpt
0 Train accuracy: 17.9576878944 Valid accuracy: 1.2097120841
1 Train accuracy: 18.484134888 Valid accuracy: 1.21816888793
2 Train accuracy: 17.7636292853 Valid accuracy: 1.16872286964
3 Train accuracy: 17.5978577317 Valid accuracy: 1.18114963442
4 Train accuracy: 17.7540071199 Valid accuracy: 1.19127305724
5 Train accuracy: 17.7149804897 Valid accuracy: 1.17401378747
6 Train accuracy: 17.7152471665 Valid accuracy: 1.17119218228
7 Train accuracy: 17.7388286337 Valid accuracy: 1.16446805828
8 Train accuracy: 17.8521551215 Valid accuracy: 1.16283751149
9 Train accuracy: 18.1611452352 Valid accuracy: 1.19158756888
10 Train accuracy: 17.5655967705 Valid accuracy: 1.151804133
11 Train accuracy: 18.1283091856 Valid accuracy: 1.18819859246
12 Train accuracy: 18.1375567389 Valid accuracy: 1.20357754889
13 Train accuracy: 17.9074340628 Valid accuracy: 1.1770505771
14 Train accuracy: 18.2283194213 Valid

In [81]:
import copy as cp
y_pred = []
with tf.Session() as sess:
    init.run()
    print('restore model')
    saver.restore(sess, "./model/vowel_transfer.ckpt")
    print(X_train1.shape)
    
    y_pred=y_hat.eval( feed_dict={is_training: False,X:X_train1})
    print(y_pred.shape)
        

restore model
INFO:tensorflow:Restoring parameters from ./model/vowel_transfer.ckpt
(150, 512)
(150, 512)


In [44]:



print(X_train1[1][8])

3.34952224459


In [82]:
import numpy as np

from scipy import io


io.savemat('y_pred.mat',{'y_pred':y_pred});
